In [ ]:
from itertools import pairwise
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from airfoil import (
    Airfoil,
    WingSegment,
    Decomposer
)
from airfoil.wing import (
    angle_degrees_to_slope,
    mirror,
    auto_piecewise,
    auto_interpolate,
    ellipse_quadrant,
    calculated_wing_cube_loading,
    calculate_wing_area,
    create_airfoil_sampler,
)
from airfoil.util.linestring_helpers import resample_shapes
import pyvista as pv

In [ ]:
leading_edge = mirror(auto_piecewise([
    (25, lambda x: 0),
    (300, lambda x: -np.tan(np.deg2rad(5))*x),
    (600, lambda x: -np.tan(np.deg2rad(3))*x),
    (700, lambda x: -np.tan(np.deg2rad(3))*x+ellipse_quadrant(100,50,x))
]))
trailing_edge = mirror(auto_piecewise([
    (25, lambda x: -100),
    (150, lambda x: np.tan(np.deg2rad(5))*x-5),
    (650, lambda x: np.tan(np.deg2rad(-2))*x),
    (700, lambda x: np.tan(np.deg2rad(-2))*x-ellipse_quadrant(50,10,x))
]))
chord = lambda x: leading_edge(x)-trailing_edge(x)
dihedral = mirror(auto_piecewise([
    (25, lambda _: 0),
    (500, lambda x: angle_degrees_to_slope(1)*x),
    (700, lambda x: angle_degrees_to_slope(15)*x),
]))
thickness = mirror(auto_interpolate([
    [  0, 0.13],
    [150, 0.13],
    [600, 0.10],
    [700, 0.12],
]))
upper_surface = lambda x: dihedral(x)+thickness(x)/2 * chord(x)
lower_surface = lambda x: dihedral(x)-thickness(x)/2 * chord(x)

washout = mirror(auto_interpolate([
    [  0,  0],
    [150,  0],
    [400, -2],
    [700, -2],
]))

airfoil_transition = mirror(auto_interpolate([
    [  0, 0],
    [400, 0],
    [500, 1],
    [700, 1],
])) 

In [ ]:
fx63137 = pd.read_csv("./data/fx63137-il.csv", dtype=pd.Float64Dtype())

shape_a = lambda x: fx63137.values.astype(np.float32) * np.array([[1,thickness(x)/0.13]])
shape_b = lambda x: Airfoil.from_naca4(
    max_camber          = 0.02,
    max_camber_position = 0.4,
    max_thickness       = thickness(x),
    chord_length        = 100,
).points
def _airfoil_shape(x:float):
    _, shape_a_resampled, shape_b_resampled = resample_shapes([
        shape_a(0),
        shape_a(x),
        shape_b(x),
    ], deflection_angle_split_deg=90)
    return airfoil_transition(x) * (shape_b_resampled - shape_a_resampled) + shape_a_resampled
    # airfoil_shape_without_control = lambda x: x*(shape_b_resampled - shape_a_resampled) + shape_a_resampled
airfoil_shape = mirror(_airfoil_shape)
ax2:plt.Axes
fig,ax= plt.subplots(figsize=(4,2))
for xx in np.linspace(0,700,20):
    shape = airfoil_shape(xx)
    ax.plot(*shape.T)


In [ ]:
sections_at = np.array([
    -25,
    25,
    150,
    300,
    400,
    500,
    600,
    630,
    660,
    680,
    690
])

In [ ]:
fig, (ax1, ax2) = plt.subplots(2,1, figsize=(15,4), sharex=True)

x = np.linspace(-700,700,400)

ax1.plot(x, np.array([leading_edge(xi) for xi in x]))
ax1.plot(x, np.array([trailing_edge(xi) for xi in x]))
ax1.set_aspect("equal")

# ax2.plot(x, np.array([upper_surface(xi) for xi in x]))
# ax2.plot(x, np.array([lower_surface(xi) for xi in x]))
# ax2.set_aspect("equal")

sar = (-sections_at)[1:][::-1].tolist()+sections_at[1:].tolist()
ax2.plot(sar, np.array([airfoil_shape(xi)[:,1].max()/100*chord(xi)+dihedral(xi) for xi in sar]))
ax2.plot(sar, np.array([airfoil_shape(xi)[:,1].min()/100*chord(xi)+dihedral(xi) for xi in sar]))
ax2.set_aspect("equal")

for section in sections_at:
    ax1.axvline(section,linestyle=":",c="r",linewidth=1)

#ax3.plot(x, np.array([washout(xi) for xi in x]))
#ax3.set_aspect("equal")
#fig.tight_layout()

In [ ]:
wing_area = calculate_wing_area(
    x,
    np.array([leading_edge(xi) for xi in x]),
    np.array([trailing_edge(xi) for xi in x]),
)
wing_area_m2 = wing_area/1000**2
wing_span = np.max(x)-np.min(x)
mean_chord = wing_area/wing_span
aspect_ratio = wing_span/mean_chord

mass_estimate = 1.0
wing_cube_loading = calculated_wing_cube_loading(mass_estimate,wing_area_m2)

print(f"""
{wing_span=:.1f}
{mean_chord=:.1f}
{aspect_ratio=:.1f}
{wing_area_m2=:.3f}
{mass_estimate=:.1f} kg
{wing_cube_loading=:.1f}
""")

In [ ]:
afs = create_airfoil_sampler(
    airfoil =lambda x: Airfoil(airfoil_shape(x)),
    leading_edge=leading_edge,
    dihedral=dihedral,
    chord=chord,
    washout=washout,
    rotation_center=lambda x: chord(x)*0.25
)

In [ ]:

fig,ax =plt.subplots(figsize=(20,10))
#for x in np.linspace(0,690,25):
for x in sections_at:
    shp = afs(x)
    shp.plot_raw(ax=ax)

In [ ]:
segments  = [
    WingSegment(
        afs(sla),
        afs(slb),
        slb-sla
    ) for sla, slb in pairwise(sections_at[1:])
]

pt = pv.Plotter()
o = 0
volume = 0
wing_meshes = []
for segment in segments:
    decomposer = Decomposer(split_angle_deg=90)
    o += segment.length/2
    msh = segment.to_mesh(decomposer)
    volume+=msh.volume*2
    wing_meshes.append(msh.translate([o,0,0]))
    wing_meshes.append(msh.scale([-1,1,1]).translate([-o,0,0]).flip_faces())
    o += segment.length/2
#pt.enable_eye_dome_lighting()
#wing_mesh = wing_meshes[0]
for m in wing_meshes:
    pt.add_mesh(m.rotate_x(-4).translate((0,0,60)))
    #wing_mesh+=m
pt.add_mesh(pv.Cylinder((0,500,0),direction=(0,1,0),radius=8,height=1000).translate((0,-180,0)))
pt.show()

In [ ]:
pt.export_gltf("./data/outputs/2025 06 20 Glider wing Design Again.gltf")

In [13]:
1+1

2